In [ ]:
!pip install streamlit pyngrok

In [ ]:
!ngrok config add-authtoken 2qRMy8d5hpeItHPYABmVLnMXJLD_8w4i6xT31sJ1vKzdrSAD

In [ ]:
%%writefile 1_Handwriting_Prediction.py
import streamlit as st
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np

model = keras.models.load_model("/content/Dyslexia_classifier.h5")

st.title("Dyslexia Classifier")
st.subheader("Upload a clear picture of the handwriting to detect signs of Dyslexia")
uploaded_pic = st.file_uploader("supported file types : jpg, jpeg, png",type = ["jpg","jpeg","png"])
if uploaded_pic:
    img = Image.open(uploaded_pic).convert("RGB")
    st.image(img,caption = "Your uploaded pic",use_container_width=True)
do_predict = st.button("Predict")
if do_predict:
    if uploaded_pic is None:
        st.write("Please upload a picture to make prediction")
    else:
        img_resized = tf.image.resize(np.array(img),(224,224))
        img_array = np.expand_dims(img_resized,axis = 0)
        img_array = img_array / 255.0

        prob = model.predict(img_array,verbose = 0)[0][0]

        if prob > 0.2:
            st.success("### Normal")
        else:
            st.error("### Dyslexia")

Writing 1_Handwriting_Prediction.py


In [ ]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
%%writefile /content/pages/2_Chat_about_dyslexia.py
import streamlit as st
from google import genai
from google.genai import types
import os

client = genai.Client(api_key = os.environ.get('GOOGLE_API_KEY'))

def get_reply(prmpt):
    system_prompt = """
    You are an educational assistant focused ONLY on dyslexia.
    - Answer user queries about dyslexia, its symptoms, detections, coping strategies, learning aids, etc.
    - If the question is unrelated to dyslexia, politely refuse and remind the user that you only handle dyslexia related queries
    """
    response = client.models.generate_content(
        model = "gemini-2.5-flash",
        config = types.GenerateContentConfig(
            system_instruction = system_prompt,
            thinking_config=types.ThinkingConfig(thinking_budget=512)
        ),
        contents = prmpt
    )
    return response.text

st.title("Chat")
st.write('Chat about anything related to dyslexia with our chatbot')

if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("Ask anything about dyslexia"):
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.spinner("thinking", show_time=True):
        bot_reply = get_reply(prompt)
    with st.chat_message("assistant"):
        st.markdown(bot_reply)

    st.session_state.messages.append({"role": "user", "content": prompt})
    st.session_state.messages.append({"role": "assistant", "content": bot_reply})

Writing /content/pages/2_Chat_about_dyslexia.py


In [ ]:
%%writefile /content/pages/3_Know_more_about_Dyslexia.py
import streamlit as st
text = """
# Understanding Dyslexia

## Introduction

Dyslexia is a common learning difference that primarily affects the ability to read, spell, and process written language. It is not related to intelligence or vision problems. Instead, it results from differences in the way the brain processes information. Dyslexia is lifelong, but with the right support, individuals can succeed academically, professionally, and personally.

---

## Key Characteristics

* **Reading difficulties**: Slow, effortful reading, frequent errors, or difficulty recognizing words.
* **Spelling challenges**: Unpredictable or inconsistent spelling patterns.
* **Writing issues**: Difficulty organizing thoughts in writing, poor grammar, or unusual handwriting.
* **Language processing**: Trouble with phonological awareness (linking sounds to letters) and rapid word retrieval.
* **Other areas**: May include challenges with sequencing, organization, or memory, but strengths in problem-solving, creativity, and verbal reasoning.

---

## Causes and Risk Factors

* **Neurological basis**: Differences in brain structure and function, especially in regions linked to reading and language.
* **Genetics**: Dyslexia often runs in families, suggesting a hereditary component.
* **Environmental factors**: Early exposure to language, quality of education, and support can influence the severity of symptoms.

---

## Signs and Symptoms

### In Children

* Delayed speech development
* Difficulty learning the alphabet or rhyming words
* Struggles with reading fluency compared to peers
* Avoidance of reading or writing tasks

### In Adolescents and Adults

* Persistent spelling and reading difficulties
* Slow reading pace
* Trouble summarizing written texts
* Difficulty learning new languages
* Lower self-confidence in academic or professional settings

---

## Diagnosis

Dyslexia is typically diagnosed through comprehensive assessments that evaluate:

* Reading and writing skills
* Phonological processing abilities
* Memory and attention
* Family and educational history

Specialists such as educational psychologists or speech-language pathologists usually conduct these evaluations.

---

## Management and Support Strategies

While dyslexia cannot be “cured,” effective support can significantly reduce its impact.

* **Educational interventions**: Structured, multisensory reading programs (e.g., Orton-Gillingham approach).
* **Classroom accommodations**: Extra time for exams, audiobooks, text-to-speech tools, or alternative formats for assignments.
* **Technology aids**: Spell-checkers, speech recognition software, and reading applications.
* **Skill development**: Targeted phonics instruction, comprehension strategies, and writing support.
* **Emotional support**: Encouragement, counseling, and building self-confidence to counteract frustration.

---

## Strengths Associated with Dyslexia

Many individuals with dyslexia display exceptional skills in:

* Creative problem-solving
* Storytelling and verbal communication
* Visual-spatial reasoning
* Innovation and out-of-the-box thinking

---

## Myths and Misconceptions

* **Myth:** Dyslexia is a vision problem.
  **Fact:** Dyslexia is related to brain-based language processing differences.

* **Myth:** People with dyslexia cannot succeed academically.
  **Fact:** With proper support, many excel in academics and careers.

* **Myth:** Dyslexia is rare.
  **Fact:** Dyslexia is one of the most common learning differences, affecting up to 10% of the population.

---

## Resources for Further Reading

* [International Dyslexia Association](https://dyslexiaida.org)
* [British Dyslexia Association](https://www.bdadyslexia.org.uk)
* [Understood.org](https://www.understood.org)

---

## Conclusion

Dyslexia is a lifelong learning difference that presents unique challenges, especially in reading and writing. However, with awareness, early intervention, and supportive strategies, individuals with dyslexia can thrive. Recognizing both the challenges and strengths associated with dyslexia helps build a more inclusive and empowering environment for all.
"""
st.markdown(text)

Writing /content/pages/3_Know_more_about_Dyslexia.py


In [11]:
process.kill()
ngrok.kill()

In [12]:
from pyngrok import ngrok
import subprocess

ngrok.kill()
public_url = ngrok.connect(8501)
print(public_url)

process = subprocess.Popen(["streamlit","run","1_Handwriting_Prediction.py"])

NgrokTunnel: "https://826d0d3e122c.ngrok-free.app" -> "http://localhost:8501"
